In [ ]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import json
import pprint
import re
import pickle
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import json

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [ ]:
!pip install datasets evaluate rouge_score transformers[torch] accelerate -U

In [ ]:
train_set = pd.read_csv('/kaggle/input/liputan6-clean-dataset/final_train_set.csv')
valid_set = pd.read_csv('/kaggle/input/liputan6-clean-dataset/final_valid_set.csv')
test_set = pd.read_csv('/kaggle/input/liputan6-clean-dataset/final_test_set.csv')

In [ ]:
train_set

,final_clean_article,combined_clean_summary
0,Kolonel Richard Ginting dinonaktifkan dari jab...,Kolonel Richard Ginting dinyatakan terlibat bi...
1,Acong ditangkap polisi karena mendalangi pencu...,Acong dipergoki rekan sesama satpam membawa ka...
2,Presiden Susilo Bambang Yudhoyono menggelar ja...,Jamuan makan malam bagi peserta KAA diadakan d...
3,Lebih dari seratus pengikut aliran Falun Gong ...,Mereka berdemonstrasi di saat para delegasi me...
4,Mantan Deputi Kepala Badan Penyehatan Perbanka...,"TMM, mantan Deputi Kepala Badan Penyehatan Per..."
...,...,...
9995,Presiden Susilo Bambang Yudhoyono telah meneke...,Presiden Susilo Bambang Yudhoyono telah menand...
9996,Sejumlah mahasiswa asal Maluku menggelar unjuk...,Sejumlah mahasiswa dari Aliansi Gerakan Peduli...
9997,"Dua kelompok massa, Front Perjuangan Pemuda In...",Massa Front Perjuangan Pemuda Indonesia (FPPI)...
9998,Wanita identik dengan keindahan dan kelembutan...,Sebanyak delapan pelukis wanita ikut serta dal...


In [ ]:
valid_set

,final_clean_article,combined_clean_summary
0,Pemerintah masih memberikan waktu dua minggu l...,Pemerintah memberikan tenggat 14 hari kepada p...
1,Diperkirakan 11 juta penduduk Indonesia positi...,Satu dari 20 orang Indonesia diperkirakan meng...
2,Aksi peledakan kantor pemerintah di Aceh terny...,Peledakan bom kembali terjadi di Aceh. Kali in...
3,Petugas Kepolisian Resor Surabaya Timur menemb...,Polres Surabaya Timur menangkap seorang penged...
4,Yogyakarta dan Bali memang gudangnya para seni...,Lima seniman Yogyakarta dan Bali menggelar pam...
...,...,...
10967,Sembilan truk yang memuat kayu balok tanpa dil...,"Pencurian kayu di Palu, Sulawesi Tengah, semak..."
10968,Dua nelayan tewas dan tiga lainnya luka-luka d...,Seorang anggota TNI Angkatan Laut menembaki ka...
10969,Kepolisian Resor Jakarta Pusat menangkap Indri...,"Indriadi, warga Kampung Tarif, Semarang, Jawa ..."
10970,Pemerintah Cina menjanjikan akses pasar yang l...,Pemerintah Cina menjanjikan akses pasar yang l...


In [ ]:
test_set

,final_clean_article,combined_clean_summary
0,Kepolisian Daerah Riau bertekad memberantas pe...,Kapolda Riau baru Brigjen Pol. Johny Yodjana b...
1,Bank Indonesia dinilai masih akan menghadapi s...,Kendati Bank Sentral AS menurunkan suku bungan...
2,Berbagai kendala menghambat pendapatan negara ...,Pemerintah bermaksud akan lebih mengandalkan s...
3,Penghapusan beberapa pasal menyangkut hak buru...,"Revisi Kepmennaker Nomor 78 Tahun 2001, dinila..."
4,"Operasi Sadar Jaya yang dilancarkan malam, se...",Polisi menangkap 32 pengunjung Diskotik Mileni...
...,...,...
10967,"Kejaksaan Agung, hingga kini, masih menunggu p...",Kejagung masih menunggu putusan hakim PN Jakse...
10968,Dewan Pimpinan Pusat Partai Kebangkitan Bangsa...,Ketua DPP PKB Khofifah Indarparawansa menuduh ...
10969,"Warga Kampung Melayu, Jakarta Timur, menggelar...","Menyambut Tahun Baru 2002, warga Kampung Melay..."
10970,"Romadhani alias Roban, penjahat kelas kakap, t...",Romadhani alias Roban tewas ditembak polisi sa...


In [ ]:
# Melakukan sampling sebanyak 1000 baris dari masing-masing dataset
train_set = train_set.sample(n=1000, random_state=42).reset_index(drop=True)
valid_set = valid_set.sample(n=300, random_state=42).reset_index(drop=True)
test_set = test_set.sample(n=300, random_state=42).reset_index(drop=True)

In [ ]:
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EncoderDecoderModel
import numpy as np

train_dataset = Dataset.from_pandas(train_set)
valid_dataset = Dataset.from_pandas(valid_set)
test_dataset = Dataset.from_pandas(test_set)

In [ ]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['final_clean_article', 'combined_clean_summary'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['final_clean_article', 'combined_clean_summary'],
        num_rows: 300
    })
    test: Dataset({
        features: ['final_clean_article', 'combined_clean_summary'],
        num_rows: 300
    })
})

In [ ]:
train_dataset = dataset_dict['train']
print(train_dataset[0])

{'final_clean_article': 'Suasana aksi unjuk rasa di depan Gedung DPR, Jalan Gatot Subroto, Senayan, Jakarta,  makin memanas. Massa membongkar paksa pagar tol dalam kota sepanjang 10 meter. Kendati demikian, aksi ini tidak sempat mengganggu arus lalu lintas tol dari arah Semanggi menuju Grogol, maupun sebaliknya. Sebelumnya, massa sempat memblokade jalan tol, namun segera digagalkan polisi, meski sempat terjadi bentrok. Aksi ini berawal dari kekecewaan pendemo yang diusir dengan penyemprot air atau water cannon, karena berusaha mendekati gerbang Gedung DPR. Pendemo yang dihujani air tak mampu membalasnya, sehingga merusak pagar jalan tol. Mereka bahkan sempat menyandera kendaraan besar, sehingga sempat terjadi kemacetan. Untuk menghindari kemacetan parah, polisi menangkap pendemo yang diduga provokator. Untuk mengantisipasi agar massa tidak sempat masuk ke dalam halaman gedung DPR, polisi kembali melakukan barikade langsung di depan pintu masuk. (ETA).', 'combined_clean_summary': 'Aksi 

In [ ]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel, EncoderDecoderModel, BertTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
import evaluate
import torch
import os
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

In [ ]:
prefix = "summary: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["final_clean_article"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)

    labels = tokenizer(text_target=examples["combined_clean_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./fine_tune_T5_model_for_Summarization",
    eval_strategy="epoch",
    learning_rate=2e-5, #menyesuakan
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir='./logs',
    fp16=True,
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.344600,2.101473,0.349600,0.190900,0.303800,0.304900,19.000000
2,1.322500,2.105323,0.349200,0.190300,0.302400,0.303500,19.000000
3,1.315000,2.109120,0.349400,0.190600,0.302600,0.303300,19.000000
4,1.183800,2.115572,0.351100,0.192500,0.303800,0.304700,19.000000
5,1.214100,2.117487,0.347700,0.188900,0.300700,0.301400,19.000000
6,1.266900,2.119836,0.350500,0.189800,0.303000,0.303500,19.000000
7,1.293100,2.127083,0.351400,0.190100,0.303000,0.303900,19.000000
8,1.232100,2.127326,0.350100,0.190300,0.303100,0.303700,19.000000
9,1.137900,2.128873,0.349800,0.190100,0.303000,0.303600,19.000000
10,1.170900,2.130416,0.349100,0.189900,0.302900,0.303700,19.000000


TrainOutput(global_step=320, training_loss=1.281679031252861, metrics={'train_runtime': 648.3748, 'train_samples_per_second': 15.423, 'train_steps_per_second': 0.494, 'total_flos': 3044789452800000.0, 'train_loss': 1.281679031252861, 'epoch': 10.0})

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

ARTICLE_TO_SUMMARIZE = """Liputan6.com, Jakarta Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC. Fungsi dari Kartu Debit Mandiri ini sangat beragam, mulai dari tarik tunai, setor tunai, transfer uang, cek saldo rekening, hingga membayar berbagai tagihan melalui mesin ATM.
Penting bagi Anda yang ingin membuka rekening tabungan di Bank Mandiri untuk memahami jenis Kartu Debit Mandiri agar tidak salah memilih. Setiap kartu debit Mandiri memiliki kelebihan dan kekurangannya masing-masing, sehingga penting bagi nasabah untuk memilih yang sesuai dengan kebutuhan dan preferensi mereka.
Dalam memilih jenis Kartu Debit Mandiri, nasabah perlu mempertimbangkan kebutuhan dan gaya hidup mereka. Apakah mereka membutuhkan manfaat tambahan seperti asuransi atau akses ke airport lounge, ataukah mereka menginginkan kartu debit yang sederhana namun praktis. Dengan mengetahui jenis Kartu Debit Mandiri yang sesuai, nasabah dapat memaksimalkan manfaat yang mereka dapatkan dari penggunaan kartu debit tersebut.
"""

# Move model to GPU
model.to('cuda')

# Encode the input text and move it to GPU
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt').to('cuda')

# Generate summary
summary_ids = model.generate(
    input_ids,
    min_length=20,
    max_length=80,
    num_beams=10,
    repetition_penalty=2.5,
    length_penalty=1.0,
    early_stopping=True,
    no_repeat_ngram_size=2,
    use_cache=True,
    do_sample=True,
    temperature=0.8,
    top_k=50,
    top_p=0.95
)

# Decode the generated summary
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC.
